##Variables


In [ ]:
sample_rate = 16000 # sample rate of the audio
n_way= 3 # number of classes per episode
n_support = 5 # number of support examples per class
n_query = 20 # number of samples per class to use as query
n_train_episodes = int(10000) # number of episodes to generate for training
n_val_episodes = 50 # number of episodes to generate for validation
num_workers = 10 # number of workers to use for data loading

In [ ]:
%%capture
!pip install torch
!pip install pytorch-lightning
!pip install numpy
!pip install --no-cache-dir --upgrade music-fsl

## FewShotLearner


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
from torch import nn
import pytorch_lightning as pl
from torchmetrics import Accuracy

from typing import Tuple
from collections import defaultdict

from music_fsl.backbone import Backbone
from music_fsl.protonet import PrototypicalNet

In [ ]:
import torch
from typing import List, Dict, Any

class ClassConditionalDataset(torch.utils.data.Dataset):

    def __getitem__(self, index: int) -> Dict[Any, Any]:
        """
        Grab an item from the dataset. The item returned must be a dictionary. 
        """
        raise NotImplementedError
    
    @property
    def classlist(self) -> List[str]:
        """
        The classlist property returns a list of class labels available in the dataset.
        This property enables users of the dataset to easily access a list of all the classes in the dataset.

        Returns:
            List[str]: A list of class labels available in the dataset. 
        """
        raise NotImplementedError

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        """
        Returns a dictionary where the keys are class labels and the values are 
        lists of indices in the dataset that belong to that class. 
        This property enables users of the dataset to easily access 
        examples that belong to specific classes. 

        Implement me!

        Returns:
            Dict[str, List[int]]: A dictionary mapping class labels to lists of dataset indices. 
        """
        raise NotImplementedError

In [ ]:
import random
import torch
import music_fsl.util as util

from typing import Tuple, Dict
class EpisodeDataset(torch.utils.data.Dataset):
    """
        A dataset for sampling few-shot learning tasks from a class-conditional dataset.

    Args:
        dataset (ClassConditionalDataset): The dataset to sample episodes from.
        n_way (int): The number of classes to sample per episode.
            Default: 5.
        n_support (int): The number of samples per class to use as support.
            Default: 5.
        n_query (int): The number of samples per class to use as query.
            Default: 20.
        n_episodes (int): The number of episodes to generate.
            Default: 100.
    """
    def __init__(self,
        dataset: ClassConditionalDataset, 
        n_way: int = 5, 
        n_support: int = 5,
        n_query: int = 20,
        n_episodes: int = 100,
    ):
        self.dataset = dataset

        self.n_way = n_way
        self.n_support = n_support
        self.n_query = n_query
        self.n_episodes = n_episodes
    
    def __getitem__(self, index: int) -> Tuple[Dict, Dict]:
        """Sample an episode from the class-conditional dataset. 

        Each episode is a tuple of two dictionaries: a support set and a query set.
        The support set contains a set of samples from each of the classes in the
        episode, and the query set contains another set of samples from each of the
        classes. The class labels are added to each item in the support and query
        sets, and the list of classes is also included in each dictionary.

        Yields:
            Tuple[Dict[str, Any], Dict[str, Any]]: A tuple containing the support
            set and the query set for an episode.
        """
        # seed the random number generator so we can reproduce this episode
        rng = random.Random(index)

        # sample the list of classes for this episode
        episode_classlist = rng.sample(self.dataset.classlist, self.n_way)

        # sample the support and query sets for this episode
        support, query = [], []
        for c in episode_classlist:
            # grab the dataset indices for this class
            all_indices = self.dataset.class_to_indices[c]
            # sample the support and query sets for this class
            indices = rng.sample(all_indices, self.n_support + self.n_query)
            items = [self.dataset[i] for i in indices]

            # add the class label to each item
            for item in items:
                item["target"] = torch.tensor(episode_classlist.index(c))

            # split the support and query sets
            support.extend(items[:self.n_support])
            query.extend(items[self.n_support:])

        # collate the support and query sets
        support = util.collate_list_of_dicts(support)
        query = util.collate_list_of_dicts(query)

        support["classlist"] = episode_classlist
        query["classlist"] = episode_classlist
        
        return support, query

    def __len__(self):
        return self.n_episodes

    def print_episode(self, support, query):
        """Print a summary of the support and query sets for an episode.

        Args:
            support (Dict[str, Any]): The support set for an episode.
            query (Dict[str, Any]): The query set for an episode.
        """
        print("Support Set:")
        print(f"  Classlist: {support['classlist']}")
        print(f"  Audio Shape: {support['audio'].shape}")
        print(f"  Target Shape: {support['target'].shape}")
        print()
        print("Query Set:")
        print(f"  Classlist: {query['classlist']}")
        print(f"  Audio Shape: {query['audio'].shape}")
        print(f"  Target Shape: {query['target'].shape}")


In [ ]:
import pytorch_lightning as pl
from torchmetrics import Accuracy
# build models
backbone = Backbone(sample_rate=sample_rate)
protonet = PrototypicalNet(backbone)

In [ ]:

class FewShotLearner(pl.LightningModule):

    def __init__(self, 
        protonet: nn.Module, 
        learning_rate: float = 0.001,
        # learning_rate: float = 1e-3,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.protonet = protonet
        self.learning_rate = learning_rate

        self.loss = nn.CrossEntropyLoss()
        self.metrics = nn.ModuleDict({
            'accuracy': Accuracy()
        })

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def step(self, batch, batch_idx, tag: str):
        support, query = batch

        logits = self.protonet(support, query)
        loss = self.loss(logits, query["target"])

        output = {"loss": loss}
        for k, metric in self.metrics.items():
            output[k] = metric(logits, query["target"])

        for k, v in output.items():
            self.log(f"{k}/{tag}", v)
        return output

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "test")

In [ ]:
learner = FewShotLearner(protonet)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'protonet' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['protonet'])`.
  rank_zero_warn(


## TinySOL

In [ ]:
import os
import pandas as pd
class TinySOL(ClassConditionalDataset):
    """
    Initialize a `TinySOL` dataset instance.

    Args:
        instruments (List[str]): A list of instruments to include in the dataset.
        duration (float): The duration of each audio clip in the dataset (in seconds).
        sample_rate (int): The sample rate of the audio clips in the dataset (in Hz).
    """

    INSTRUMENTS = [
        'Bassoon', 'Viola', 'Trumpet in C', 'Bass Tuba',
        'Alto Saxophone', 'French Horn', 'Violin', 
        'Flute', 'Contrabass', 'Trombone', 'Cello', 
        'Clarinet in Bb', 'Oboe', 'Accordion'
    ]

    def __init__(self, 
            instruments: List[str] = None,
            duration: float = 1.0, 
            sample_rate: int = 16000,
            dataset_path: str = None
        ):
        if instruments is None:
            instruments = self.INSTRUMENTS

        self.instruments = instruments
        self.duration = duration
        self.sample_rate = sample_rate

        # initialize the tinysol dataset
        metadata = os.path.join(dataset_path, 'TinySOL_metadata.csv')
        df = pd.read_csv(metadata) 
        paths = df.iloc[:, 0].apply(lambda x: os.path.join(dataset_path, x))
        paths = paths.values.tolist()
        labels = df.iloc[:, 4].values

        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.instruments, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        self.tracks = []
        for path, label in zip(paths, labels):
            if label in self.instruments:
              self.tracks.append([path, label])

    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

        return data

    def __len__(self) -> int:
        return len(self.tracks)

In [ ]:
TRAIN_INSTRUMENTS = [
    'French Horn', 
    'Violin', 
    'Flute', 
    'Contrabass', 
    'Trombone', 
    'Cello', 
    'Clarinet in Bb', 
    'Oboe',
    'Accordion'
]

TEST_INSTRUMENTS = [
    'Bassoon',
    'Viola',
    'Trumpet in C',
    'Bass Tuba',
    'Alto Saxophone'
]

## GoodSounds

In [ ]:
class GoodSounds(ClassConditionalDataset):
    """
    Initialize a `GoodSounds Dataset Loader` dataset instance.
    
    Args:
        instruments (List[str]): A list of instruments to include in the dataset.
        duration (float): The duration of each audio clip in the dataset (in seconds).
        sample_rate (int): The sample rate of the audio clips in the dataset (in Hz).
        dataset - loaded mirdata.dataset
    """

    INSTRUMENTS = [
        'flute', 'cello', 'clarinet', 'trumpet', 'violin', 'sax_alto', 'sax_tenor', 'sax_baritone', 'sax_soprano', 'oboe', 'piccolo', 'bass'
    ]

    def __init__(self, 
            instruments: List[str] = None,
            duration: float = 1.0, 
            sample_rate: int = 16000,
            dataset_path: str = None
        ):
        if instruments is None:
            instruments = self.INSTRUMENTS

        self.instruments = instruments  
        self.duration = duration
        self.sample_rate = sample_rate
        self.dataset_path = dataset_path

        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.INSTRUMENTS, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        self.tracks = []
        for dir in os.listdir(self.dataset_path):
            ins = dir.split('_')[0]
            if ins in self.instruments:
                for subdir_dir, dirs_dir, files_dir in os.walk(os.path.join(self.dataset_path, dir, 'neumann')):
                    for file in files_dir:
                        if file.endswith('.wav'):
                            if librosa.get_duration(filename=os.path.join(self.dataset_path, dir, 'neumann', file)) >= duration:
                                self.tracks.append([os.path.join(self.dataset_path, dir, 'neumann', file), ins])
            else:
                ins = f'{ins}_{dir.split("_")[1]}'
                if ins in self.instruments:
                    for subdir_dir, dirs_dir, files_dir in os.walk(os.path.join(self.dataset_path, dir, 'neumann')):
                        for file in files_dir:
                            if file.endswith('.wav'):
                                if librosa.get_duration(filename=os.path.join(self.dataset_path, dir, 'neumann', file)) >= duration:
                                    self.tracks.append([os.path.join(self.dataset_path, dir, 'neumann', file), ins])


    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

        return data

    def __len__(self) -> int:
        return len(self.tracks)

In [ ]:

TRAIN_INSTRUMENTS = [
       'cello', 'clarinet', 'violin', 'sax_alto', 'sax_baritone', 'sax_soprano', 'piccolo',
    ]

TEST_INSTRUMENTS = [
        'flute', 'trumpet', 'sax_tenor', 'oboe', 'bass'
    ]


##Medley_solos_db

In [ ]:
import os
import pandas as pd

class MedleySolosDb(ClassConditionalDataset):
    """
    Initialize a `Medley-solos-DB Dataset Loader` dataset instance.
    
     Each of these clips contains a single instrument among a taxonomy of eight:

        0. clarinet,
        1. distorted electric guitar,
        2. female singer,
        3. flute,
        4. piano,
        5. tenor saxophone,
        6. trumpet, and
        7. violin.

    Args:
        instruments (List[str]): A list of instruments to include in the dataset.
        duration (float): The duration of each audio clip in the dataset (in seconds).
        sample_rate (int): The sample rate of the audio clips in the dataset (in Hz).
    """

    INSTRUMENTS = [
        'clarinet',
        'distorted electric guitar',
        'female singer',
        'flute',
        'piano',
        'tenor saxophone',
        'trumpet',
        'violin'
    ]

    def __init__(self, 
            instruments: List[str] = None,
            duration: float = 1.0, 
            sample_rate: int = 16000,
            dataset_path: str = None,
        ):
        if instruments is None:
            instruments = self.INSTRUMENTS

        self.instruments = instruments  
        self.duration = duration
        self.sample_rate = sample_rate

        metadata = os.path.join(dataset_path, 'Medley-solos-DB_metadata.csv')
        df = pd.read_csv(metadata) 

        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.INSTRUMENTS, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        self.tracks = []
        for index, row in df.iterrows():
            if row['instrument'] in self.instruments:
                file_name = f'Medley-solos-DB_{row["subset"]}-{row["instrument_id"]}_{row["uuid4"]}.wav.wav'
                wav_file = os.path.join(dataset_path, 'Medley-solos-DB', file_name)
                self.tracks.append([wav_file, row['instrument']])


    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

        return data

    def __len__(self) -> int:
        return len(self.tracks)

In [ ]:
TRAIN_INSTRUMENTS = [
        'flute', 'piano', 'tenor saxophone', 'trumpet', 'violin'
    ]

TEST_INSTRUMENTS = [
        'clarinet', 'distorted electric guitar', 'female singer',
    ]


## IRMAS

In [ ]:
from collections import defaultdict
import mirdata
import music_fsl.util as util
from typing import List, Dict
import os
import sys


class IRMAS(ClassConditionalDataset):
    """
    Initialize a `IRMAS` dataset instance.

    Args:
        instruments (List[str]): A list of instruments to include in the dataset.
        duration (float): The duration of each audio clip in the dataset (in seconds).
        sample_rate (int): The sample rate of the audio clips in the dataset (in Hz).
    """

    INSTRUMENTS = [
        'Cello', 'Clarinet', 'Flute', 'Acoustic guitar',
        'Electric guitar', 'Organ', 'Piano', 
        'Saxophone', 'Trumpet', 'Violin', 'Human singing voice'
    ]

    INSTRUMENTS_KEY = {
        'cel': 'Cello', 'cla' : 'Clarinet', 'flu' : 'Flute', 'gac' : 'Acoustic guitar',
        'gel' : 'Electric guitar', 'org' : 'Organ', 'pia' : 'Piano', 
        'sax' : 'Saxophone', 'tru' : 'Trumpet', 'vio' : 'Violin', 'voi' : 'Human singing voice'
    }
    def __init__(self, 
            instruments: List[str] = None,
            instruments_key: Dict[str, str] = None,
            duration: float = 1.0, 
            sample_rate: int = 16000,
            dataset_path: str = 'irmas',
            val = False,
        ):
        if instruments_key is None:
            instruments_key = self.INSTRUMENTS_KEY

        if instruments is None:
            instruments = self.INSTRUMENTS

        if val == False and dataset_path == 'irmas':
            dir_path = os.path.dirname(os.path.realpath(__file__))
            dataset_path = os.path.join(os.path.dirname(dir_path), dataset_path)
        

        self.instruments = instruments
        self.instruments_key = instruments_key
        self.duration = duration
        self.sample_rate = sample_rate

        # initialize IRMAS path 
        if val == False:
          if os.path.exists(dataset_path):
            self.dataset_path = dataset_path
          else:
            sys.exit("Dataset path does not exist")
        
        if val:
          self.dataset_path = dataset_path

        # make sure the instruments passed in are valid
        for instrument in instruments:
            assert instrument in self.INSTRUMENTS, f"{instrument} is not a valid instrument"

        # load all tracks for this instrument
        if val:
            self.load_all_tracks_val()
        else:
            self.load_all_tracks()


    def load_all_tracks(self):
        self.tracks = []
        for subdir, dirs, files in os.walk(self.dataset_path):
            for dir in dirs:
                if dir in self.instruments_key.keys():
                    for subdir_dir, dirs_dir, files_dir in os.walk(os.path.join(subdir, dir)):
                        for file in files_dir:
                            self.tracks.append([os.path.join(subdir, dir, file), self.instruments_key[dir]])

    def load_all_tracks_val(self):
        self.tracks = []
        for paths in self.dataset_path:
          for subdir, dirs, files in os.walk(paths, topdown=True):
              for file in sorted(files)[::2]:
                  f = open(os.path.join(subdir, file), "rb")
                  tags = f.readlines()
                  for tag in tags:
                      tag = tag[:3].decode("utf-8") 
                      f.close()
                      if tag in self.instruments_key.keys():
                          wav_file = os.path.join(subdir, file).replace('.txt', '.wav')
                          if os.path.exists(wav_file):
                            self.tracks.append([wav_file, self.instruments_key[tag]])

    @property
    def classlist(self) -> List[str]:
        return self.instruments

    @property
    def class_to_indices(self) -> Dict[str, List[int]]:
        # cache it in self._class_to_indices 
        # so we don't have to recompute it every time
        if not hasattr(self, "_class_to_indices"):
            self._class_to_indices = defaultdict(list)
            for i, track in enumerate(self.tracks):
                self._class_to_indices[track[1]].append(i)

        return self._class_to_indices

    def __getitem__(self, index) -> Dict:
        # load the track for this index
        track = self.tracks[index]

        # load the excerpt
        data = util.load_excerpt(track[0], self.duration, self.sample_rate)
        data["label"] = track[1]

        return data

    def __len__(self) -> int:
        return len(self.tracks)

In [ ]:
TRAIN_INSTRUMENTS = [
        'Flute', 'Organ', 'Saxophone',
        'Trumpet', 'Violin', 'Electric guitar'
    ]

TRAIN_INSTRUMENTS_KEY = {
        'flu' : 'Flute', 'org' : 'Organ',
        'sax' : 'Saxophone', 'tru' : 'Trumpet', 'vio' : 'Violin', 'gel' : 'Electric guitar'
    }

TEST_INSTRUMENTS = [
        'Cello', 'Piano', 'Clarinet', 'Acoustic guitar', 'Human singing voice'
    ]

TEST_INSTRUMENTS_KEY = {
        'cel': 'Cello', 'gac' : 'Acoustic guitar', 'pia' : 'Piano',  'voi' : 'Human singing voice', 'cla' : 'Clarinet'
    }

#Data analysis

Prepare choosen models (for 3-way and 5-way)

In [ ]:
!unzip /content/irmas_3_5_1000.zip -d /content/3_5_irmas

Archive:  /content/irmas_3_5_1000.zip
   creating: /content/3_5_irmas/content/logs/
   creating: /content/3_5_irmas/content/logs/version_1/
  inflating: /content/3_5_irmas/content/logs/version_1/events.out.tfevents.1674778842.e98b3886b3c5.5426.1  
   creating: /content/3_5_irmas/content/logs/version_1/checkpoints/
  inflating: /content/3_5_irmas/content/logs/version_1/checkpoints/epoch=0-step=1000.ckpt  
  inflating: /content/3_5_irmas/content/logs/version_1/fit-profile.txt.txt  
  inflating: /content/3_5_irmas/content/logs/version_1/hparams.yaml  
   creating: /content/3_5_irmas/content/logs/version_2/
  inflating: /content/3_5_irmas/content/logs/version_2/events.out.tfevents.1674788703.e98b3886b3c5.5426.2  
   creating: /content/3_5_irmas/content/logs/version_2/checkpoints/
  inflating: /content/3_5_irmas/content/logs/version_2/checkpoints/epoch=0-step=1000.ckpt  
  inflating: /content/3_5_irmas/content/logs/version_2/fit-profile.txt.txt  
  inflating: /content/3_5_irmas/content/logs

In [ ]:
!unzip /content/irmas_5_5_1000.zip -d /content/5_5_irmas

Archive:  /content/irmas_5_5_1000.zip
   creating: /content/5_5_irmas/content/logs/
   creating: /content/5_5_irmas/content/logs/version_0/
  inflating: /content/5_5_irmas/content/logs/version_0/hparams.yaml  
  inflating: /content/5_5_irmas/content/logs/version_0/events.out.tfevents.1674778509.e98b3886b3c5.5426.0  
   creating: /content/5_5_irmas/content/logs/version_1/
  inflating: /content/5_5_irmas/content/logs/version_1/events.out.tfevents.1674778842.e98b3886b3c5.5426.1  
   creating: /content/5_5_irmas/content/logs/version_1/checkpoints/
  inflating: /content/5_5_irmas/content/logs/version_1/checkpoints/epoch=0-step=1000.ckpt  
  inflating: /content/5_5_irmas/content/logs/version_1/fit-profile.txt.txt  
  inflating: /content/5_5_irmas/content/logs/version_1/hparams.yaml  


In [ ]:
%%capture
!pip install "torchmetrics==0.10.2" 
!pip install tqdm

In [ ]:
from pathlib import Path

import numpy as np
import torch
import tqdm
from torchmetrics import Accuracy

from music_fsl.util import dim_reduce, embedding_plot, batch_device

In [ ]:
checkpoint_path_5 = "/content/5_5_irmas/content/logs/version_1/checkpoints/epoch=0-step=1000.ckpt"
checkpoint_path_3 = "/content/3_5_irmas/content/logs/version_2/checkpoints/epoch=0-step=1000.ckpt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sample_rate = 16000

In [ ]:
protonet = PrototypicalNet(Backbone(sample_rate))
learner_5 = FewShotLearner.load_from_checkpoint(checkpoint_path_5, protonet=protonet)
learner_5.eval()
learner_5 = learner_5.to(DEVICE)

learner_3 = FewShotLearner.load_from_checkpoint(checkpoint_path_3, protonet=protonet)
learner_3.eval()
learner_3 = learner_3.to(DEVICE)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'protonet' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['protonet'])`.
  rank_zero_warn(


In [ ]:
# initialize the datasets

dataset_withcut = IRMAS(
    instruments=TEST_INSTRUMENTS, 
    instruments_key = TEST_INSTRUMENTS_KEY,
    sample_rate=sample_rate,
    dataset_path = '/content/drive/MyDrive/irmas/training/IRMAS-TrainingData'
)

In [ ]:
n_query = 15

In [ ]:
# load our evaluation data
n_way=3 
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

##Metric

In [ ]:
metric_3 = Accuracy(num_classes=n_way, average="samples")
metric_5 = Accuracy(num_classes=n_way, average="samples")

###3waytrain_3waytest


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table_3 = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner_3.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric_3(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table_3.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table_3.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels([TEST_INSTRUMENTS)

plt.show()


In [ ]:
total_acc = metric_3.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

###5waytrain_3waytest


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table_5 = []
y_act_5, y_pred_5 = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner_3.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act_5.extend(query_target)
    outputs = softmax(logits)
    _, predicted = torch.max(outputs, 1)
    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred_5.extend(predicted_s)
    acc = metric_5(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table_5.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table_5.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix_5 = metrics.confusion_matrix(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix_5, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric_5.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

##For 5-way train

In [ ]:
# load our evaluation data
n_way=5
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

In [ ]:
metric_3 = Accuracy(num_classes=n_way, average="samples")
metric_5 = Accuracy(num_classes=n_way, average="samples")

###3waytrain_5waytest

In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table_3 = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner_3.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric_3(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table_3.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table_3.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric_3.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

###5waytrain_5waytest


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table_5 = []
y_act_5, y_pred_5 = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner_3.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act_5.extend(query_target)
    outputs = softmax(logits)
    _, predicted = torch.max(outputs, 1)
    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred_5.extend(predicted_s)
    acc = metric_5(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table_5.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table_5.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix_5 = metrics.confusion_matrix(y_act_5, y_pred_5, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix_5, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric_5.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

#Goodsounds


In [ ]:
!unzip /content/filegoodsounds01.zip -d /content/Goodsouns

In [ ]:

checkpoint_path = "/content/Goodsouns/content/logs/version_1/checkpoints/epoch=0-step=1000.ckpt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sample_rate = 16000

protonet = PrototypicalNet(Backbone(sample_rate))
learner = FewShotLearner.load_from_checkpoint(checkpoint_path, protonet=protonet)
learner.eval()
learner = learner.to(DEVICE)

In [ ]:
# load our evaluation data
n_way=3
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

##Metric

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##3


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

##For 5-way train

In [ ]:
# load our evaluation data
n_way=5
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##5


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

#iRMAS

In [ ]:
!unzip /content/irmas_5_5_1000.zip -d /content/IRMAS

In [ ]:

checkpoint_path = "/content/IRMAS/content/logs/version_1/checkpoints/epoch=0-step=1000.ckpt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sample_rate = 16000

protonet = PrototypicalNet(Backbone(sample_rate))
learner = FewShotLearner.load_from_checkpoint(checkpoint_path, protonet=protonet)
learner.eval()
learner = learner.to(DEVICE)

In [ ]:
# load our evaluation data
n_way=3 
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

##Metric

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##3


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

##For 5-way train

In [ ]:
# load our evaluation data
n_way=5
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##5


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

#TinySOL


In [ ]:
!unzip /content/tinysol_5_5_1000_001_1s.zip -d /content/TINYSOL

Archive:  /content/tinysol_5_5_1000_001_1s.zip
   creating: /content/TINYSOL/content/logs/
   creating: /content/TINYSOL/content/logs/version_0/
  inflating: /content/TINYSOL/content/logs/version_0/events.out.tfevents.1674613023.4f5b1a403261.2978.0  
  inflating: /content/TINYSOL/content/logs/version_0/hparams.yaml  
  inflating: /content/TINYSOL/content/logs/version_0/fit-profile.txt.txt  
   creating: /content/TINYSOL/content/logs/version_0/checkpoints/
  inflating: /content/TINYSOL/content/logs/version_0/checkpoints/epoch=0-step=1000.ckpt  


In [ ]:

checkpoint_path = "/content/TINYSOL/content/logs/version_0/checkpoints/epoch=0-step=1000.ckpt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sample_rate = 16000

protonet = PrototypicalNet(Backbone(sample_rate))
learner = FewShotLearner.load_from_checkpoint(checkpoint_path, protonet=protonet)
learner.eval()
learner = learner.to(DEVICE)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'protonet' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['protonet'])`.
  rank_zero_warn(


In [ ]:
# load our evaluation data
n_way=3 
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

##Metric

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##3


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-86-3329f8b0b161>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = softmax(logits)
Episode 99 // Accuracy: 0.60: 100%|██████████| 100/100 [02:39<00:00,  1.60s/it]


In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

##For 5-way train

In [ ]:
# load our evaluation data
n_way=5
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##5


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=TEST_INSTRUMENTS))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=TEST_INSTRUMENTS)

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(TEST_INSTRUMENTS)
ax.yaxis.set_ticklabels(TEST_INSTRUMENTS)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

#MadleySolosDB

In [ ]:
!unzip /content/medley_solos_db_3_5_001_1000.zip -d /content/MADLEY

In [ ]:

checkpoint_path = "/content/MADLEY/content/logs/version_4/checkpoints/epoch=0-step=1000.ckpt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sample_rate = 16000

protonet = PrototypicalNet(Backbone(sample_rate))
learner = FewShotLearner.load_from_checkpoint(checkpoint_path, protonet=protonet)
learner.eval()
learner = learner.to(DEVICE)

In [ ]:
# load our evaluation data
n_way=3 
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

##Metric

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##3


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass']))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass']))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])
ax.yaxis.set_ticklabels(['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")

##For 5-way train

In [ ]:
# load our evaluation data
n_way=5
n_support=5
n_episodes = 100 

test_episodes = EpisodeDataset(
    dataset=dataset_withcut, 
    n_way=n_way, 
    n_support=n_support,
    n_query=n_query, 
    n_episodes=n_episodes
)

In [ ]:
metric = Accuracy(num_classes=n_way, average="samples")

##5


In [ ]:
# collect all the embeddings in the test set
# so we can plot them later
from torch import nn
softmax = nn.Softmax()

embedding_table = []
y_act, y_pred = [], []

pbar = tqdm.tqdm(range(len(test_episodes)))
for episode_idx in pbar:
    support, query = test_episodes[episode_idx]

    # move all tensors to cuda if necessary
    batch_device(support, DEVICE)
    batch_device(query, DEVICE)

    # get the embeddings
    logits = learner.protonet(support, query)

    # compute the accuracy
    query_target = [support["classlist"][i] for i in query["target"]]
    y_act.extend(query_target)

    outputs = softmax(logits)

    _, predicted = torch.max(outputs, 1)

    predicted_s = [support["classlist"][i] for i in predicted]
    y_pred.extend(predicted_s)
    acc = metric(logits, query["target"])
    pbar.set_description(f"Episode {episode_idx} // Accuracy: {acc.item():.2f}")
    # add all the support and query embeddings to our records
    for subset_idx, subset in enumerate((support, query)):
        for emb, label in zip(subset["embeddings"], subset["target"]):
            embedding_table.append({
                "embedding": emb.detach().cpu().numpy(),
                "label": support["classlist"][label],
                "marker": ("support", "query")[subset_idx], 
                "episode_idx": episode_idx
            })
        
    # also add the prototype embeddings to our records
    for class_idx, emb in enumerate(support["prototypes"]):
        embedding_table.append({
            "embedding": emb.detach().cpu().numpy(),
            "label": support["classlist"][class_idx],
            "marker": "prototype", 
            "episode_idx": episode_idx
        })

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass']))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass']))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cf_matrix = metrics.confusion_matrix(y_act, y_pred, labels=['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])

ax = sns.heatmap(cf_matrix, annot=True, )

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])
ax.yaxis.set_ticklabels(['flute', 'trumpet', 'sax_tenor', 'sax_alto', 'bass'])

## Display the visualization of the Confusion Matrix.
plt.show()


In [ ]:
# compute the total accuracy across all episodes
total_acc = metric.compute()
print(f"Total accuracy, averaged across all episodes: {total_acc:.2f}")